In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We will install byaldi from source to get started.

In [2]:
!git clone https://github.com/AnswerDotAI/byaldi.git

Cloning into 'byaldi'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 176 (delta 3), reused 3 (delta 3), pack-reused 164 (from 1)
Receiving objects: 100% (176/176), 1.96 MiB | 20.51 MiB/s, done.
Resolving deltas: 100% (92/92), done.


In [3]:
!sudo apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 0s (2,045 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 12359

In [4]:
%cd byaldi

/content/byaldi


In [5]:
!pip install -e "."

Obtaining file:///content/byaldi
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.9/517.9 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta

In [6]:
!pip install -q pdf2image

We should initialize `RAGMultiModalModel` object with a ColPali model from Hugging Face. By default this model uses GPU but we are going to have Qwen2-VL in the same GPU so we are loading this in CPU for now.

In [7]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

Device: cuda


In [8]:
DOCUMENT_FILEPATH = "/content/drive/MyDrive/NAACL_Research_Urdu/data/constitution_pdfs/constitution_english_reduced.pdf"
EMBEDDINGS_FILEPATH = "/content/drive/MyDrive/NAACL_Research_Urdu/data/colpali_multimodal_embeddings"
DF_FILEPATH = '/content/drive/MyDrive/NAACL_Research_Urdu/data/augmented_datasets/qa_dataset_eng_v1_with_page_span.csv'

In [9]:
from pdf2image import convert_from_path

images = convert_from_path(DOCUMENT_FILEPATH)

In [10]:
from byaldi import RAGMultiModalModel

# RAG = RAGMultiModalModel.from_pretrained("vidore/colpali")
RAG = RAGMultiModalModel.from_index(EMBEDDINGS_FILEPATH)

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(


...

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

/content/byaldi/byaldi/colpali.py:138: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.indexed_embeddings.extend(torch.load(file))


We can directly index our document using RAG, simply passing pdf file path is enough.

In [11]:
# RAG.index(
#     input_path=DOCUMENT_FILEPATH,
#     index_name="image_index", # index will be saved at index_root/index_name/
#     store_collection_with_index=False,
#     overwrite=True
# )

In [12]:
def retrieve_page_numbers(query, k=3):
    results = RAG.search(query, k)
    return tuple(result['page_num'] for result in results)

Now let's retrieve!

In [13]:
text_query = "what are the conditions for being appointed as a supreme court judge?"
retrieve_page_numbers(text_query)

(96, 118, 110)

In [14]:
import pandas as pd
import ast
from tqdm import tqdm
tqdm.pandas()

In [58]:
df = pd.read_csv(DF_FILEPATH)

# convert string to tuple
df['Chunk_Page_Span'] = df['Chunk_Page_Span'].progress_apply(ast.literal_eval)
df

100%|██████████| 621/621 [00:00<00:00, 50428.12it/s]


,Question,Answer,Answer Chunk Index,Chunk_Page_Span
0,What type of government does Pakistan have?,Pakistan is a Federal Republic known as the Is...,0,"(1, 1)"
1,Can new states or areas become part of Pakistan?,"Yes, the Majlis-e-Shoora (Parliament) may admi...",0,"(1, 1)"
2,What is the designated religion of Pakistan?,Islam shall be the State religion of Pakistan.,1,"(1, 1)"
3,What has been made a substantive part of the C...,The principles and provisions set out in the O...,2,"(1, 1)"
4,What is the State's responsibility regarding e...,The State shall ensure the elimination of all ...,3,"(2, 2)"
...,...,...,...,...
616,Who is responsible for handling accounts that ...,The Auditor-General is responsible for handlin...,309,"(174, 174)"
617,What powers does the Auditor-General have rega...,The Auditor-General has the same powers and fu...,309,"(174, 174)"
618,Will existing taxes continue to be collected a...,"Yes, all taxes and fees levied under any law i...",310,"(174, 175)"
619,When was the Proclamation of Emergency mention...,The Proclamation of Emergency was originally i...,311,"(175, 175)"


In [59]:
max_len = df['Chunk_Page_Span'].apply(lambda x: x[1] - x[0]).max()
print(f"The largest chunk spans: {max_len} pages")

The largest chunk spans: 4 pages


In [ ]:
def split_rets(rets):
    return [rets[:i] for i in range(1, len(rets) + 1)]

In [60]:
results = df['Question'].progress_apply(lambda question: retrieve_page_numbers(question, 5))

100%|██████████| 621/621 [02:35<00:00,  3.98it/s]


In [70]:
expanded_responses = results.progress_apply(lambda x: split_rets(x))
expanded_df = pd.DataFrame(expanded_responses.tolist(), columns=['TOP_1', 'TOP_2', 'TOP_3', 'TOP_4', 'TOP_5'])
expanded_df.head()

100%|██████████| 621/621 [00:00<00:00, 98059.74it/s]


,TOP_1,TOP_2,TOP_3,TOP_4,TOP_5
0,"(72,)","(72, 4)","(72, 4, 10)","(72, 4, 10, 17)","(72, 4, 10, 17, 152)"
1,"(1,)","(1, 76)","(1, 76, 73)","(1, 76, 73, 169)","(1, 76, 73, 169, 18)"
2,"(10,)","(10, 1)","(10, 1, 11)","(10, 1, 11, 157)","(10, 1, 11, 157, 156)"
3,"(1,)","(1, 37)","(1, 37, 66)","(1, 37, 66, 167)","(1, 37, 66, 167, 38)"
4,"(2,)","(2, 17)","(2, 17, 16)","(2, 17, 16, 7)","(2, 17, 16, 7, 15)"


In [71]:
df = pd.concat([df, expanded_df], axis=1)

In [72]:
df

,Question,Answer,Answer Chunk Index,Chunk_Page_Span,TOP_1,TOP_2,TOP_3,TOP_4,TOP_5
0,What type of government does Pakistan have?,Pakistan is a Federal Republic known as the Is...,0,"(1, 1)","(72,)","(72, 4)","(72, 4, 10)","(72, 4, 10, 17)","(72, 4, 10, 17, 152)"
1,Can new states or areas become part of Pakistan?,"Yes, the Majlis-e-Shoora (Parliament) may admi...",0,"(1, 1)","(1,)","(1, 76)","(1, 76, 73)","(1, 76, 73, 169)","(1, 76, 73, 169, 18)"
2,What is the designated religion of Pakistan?,Islam shall be the State religion of Pakistan.,1,"(1, 1)","(10,)","(10, 1)","(10, 1, 11)","(10, 1, 11, 157)","(10, 1, 11, 157, 156)"
3,What has been made a substantive part of the C...,The principles and provisions set out in the O...,2,"(1, 1)","(1,)","(1, 37)","(1, 37, 66)","(1, 37, 66, 167)","(1, 37, 66, 167, 38)"
4,What is the State's responsibility regarding e...,The State shall ensure the elimination of all ...,3,"(2, 2)","(2,)","(2, 17)","(2, 17, 16)","(2, 17, 16, 7)","(2, 17, 16, 7, 15)"
...,...,...,...,...,...,...,...,...,...
616,Who is responsible for handling accounts that ...,The Auditor-General is responsible for handlin...,309,"(174, 174)","(173,)","(173, 89)","(173, 89, 88)","(173, 89, 88, 41)","(173, 89, 88, 41, 87)"
617,What powers does the Auditor-General have rega...,The Auditor-General has the same powers and fu...,309,"(174, 174)","(173,)","(173, 89)","(173, 89, 172)","(173, 89, 172, 45)","(173, 89, 172, 45, 150)"
618,Will existing taxes continue to be collected a...,"Yes, all taxes and fees levied under any law i...",310,"(174, 175)","(166,)","(166, 174)","(166, 174, 4)","(166, 174, 4, 86)","(166, 174, 4, 86, 154)"
619,When was the Proclamation of Emergency mention...,The Proclamation of Emergency was originally i...,311,"(175, 175)","(174,)","(174, 138)","(174, 138, 136)","(174, 138, 136, 139)","(174, 138, 136, 139, 137)"


In [73]:
df.to_csv(f'/content/drive/MyDrive/NAACL_Research_Urdu/data/augmented_datasets/qa_dataset_eng_v1_with_page_span_and_multimodal_top_k.csv')

In [112]:
def get_page_range_tuple(span):
    n_pages = span[1] - span[0] + 1

    if n_pages == 1:
        return (span[1],)

    return tuple(range(span[0], span[0] + n_pages))

def calculate_weighted_score(gt_pages, preds):
    preds_set = set(preds)
    gt_pages_set = set(gt_pages)

    correct_count = len(preds_set & gt_pages_set)  # Intersection

    if len(preds) > 1:
        return correct_count / len(gt_pages)

    return correct_count

def calculate_non_weighted_score(gt_pages, preds):
    preds_set = set(preds)
    gt_pages_set = set(gt_pages)

    if preds_set & gt_pages_set:
        return 1

    return 0

def calculate_top_k(df, k, weighted=True):
    score = 0

    for gt_span, preds in zip(df['Chunk_Page_Span'], df[f'TOP_{k}']):
        gt_pages = get_page_range_tuple(gt_span)

        if weighted:
            score += calculate_weighted_score(gt_pages, preds)
        else:
            score += calculate_non_weighted_score(gt_pages, preds)

    return score / len(df)

In [113]:
for k in range(1, 6):
    print(f"Weighted Top {k}: {calculate_top_k(df, k):.2f}%")

Weighted Top 1: 0.39%
Weighted Top 2: 0.39%
Weighted Top 3: 0.46%
Weighted Top 4: 0.51%
Weighted Top 5: 0.54%


In [114]:
for k in range(1, 6):
    print(f"Non-Weighted Top {k}: {calculate_top_k(df, k, weighted=False):.2f}%")

Non-Weighted Top 1: 0.39%
Non-Weighted Top 2: 0.55%
Non-Weighted Top 3: 0.61%
Non-Weighted Top 4: 0.65%
Non-Weighted Top 5: 0.66%
